# Szyfry blokowe

## AKA

Przed wykonaniem szyfrowania i deszyfrowania strony muszą być wyposażone w klucz tajny. Jeżeli współdzielona tajemnica jest tylko hasłem o niskiej entropii to należy przeprowadzić procedurę *key streching*. Oprócz współdzielonego hasła jej parametrem jest współdzielona i jawna liczba losowa *salt*. Od strony teoretycznej procedury te zostaną omówione przy innej okazji. Tutaj zostanie przedstawiona tylko procedura która realizuje to zadanie. Procedurę tę muszą wykonać nadawca i odbiorca.

In [1]:
import binascii
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
# Key stretching
password = b'password123'
salt = get_random_bytes(16)
# salt = b'\x00' * 16
secret_key = PBKDF2(password, salt, 32, count=100000, hmac_hash_module=SHA256)
print("Salt= ", binascii.hexlify(salt))
print("Secret Key= ", binascii.hexlify(secret_key))

Salt=  b'c21a756a4caf6431e47b9a540f616481'
Secret Key=  b'd33f57cb8b61449062f8ec37013be379ca515c5c8bbf78cff330eb3efaaf6c94'


## Tryb CBC

### Szyfrowanie

Szyfrowanie obejmuje 
* przekształcenie hasła uzytkownika w klucz kryptograficzny o długości wymaganej przez algorytm,
* przygotowanie danych dla trybu pracy,
* preprocessing wiadomości (padding),

In [3]:
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
from Crypto.Util import Padding
# Prepare cipher
iv = get_random_bytes(AES.block_size)
cipher = AES.new(secret_key, AES.MODE_CBC, iv)
# Prepare message
message = 'To jest text do zaszyfrowania i jeszcze trochę aby wyszły dwa bloki'
plaintext = Padding.pad(message.encode('utf-8'),AES.block_size,'pkcs7')
ciphertext = cipher.encrypt(plaintext)
message = iv + ciphertext
# Send encrypted data
print("Recipient receives:")
print("IV= ", binascii.hexlify(iv))
print("Ciphertext= ", binascii.hexlify(ciphertext))

Recipient receives:
IV=  b'b4d1e607820be44c13e683c414fd759a'
Ciphertext=  b'5e6c3eded51677bc3b6aec1982011b5b47f2114c6342ea7fa83f95e47bd2069e354a3a8acfb1adaccc2bb6ff5de66a7c766d8cca707b25ae884b942c0db063f32065fa6d782c73214d25f9fd572e7337'


### Deszyfrowanie

Odbiorca z przeprowadzonej wcześniej procedury AKA ma odtworzony *secret_key*.
Deszyfrowanie składa się z następujących kroków:
* odzyskanie wartości *iv* zawartej w komunikacie i zainicjowanie szyfru,
* konwersja szyfrogramu do tekstu jawnego,
* usuniecię uzupełnienia dodanego przed zaszyfrowaniem.

In [22]:
decrypt_key = PBKDF2(b'password123', salt, 32, count=100000, hmac_hash_module=SHA256)
iv = message[:16]
ciphertext = message[16:]
# Prepare cipher
cipher = AES.new(decrypt_key, AES.MODE_CBC, iv)
# Decrypt ciphertext
received_plaintext = cipher.decrypt(ciphertext)
# Remove padding
received_message = Padding.unpad(received_plaintext,AES.block_size,'pkcs7').decode('utf-8')
# Send encrypted data
print("Recipient decodes: ")
print(received_message)

Recipient decodes: 
To jest text do zaszyfrowania i jeszcze trochę aby wyszły dwa bloki
